In [449]:
import pandas as pd
import numpy as np

In [558]:
hosp = pd.read_csv('Hospitals_geocoded.csv')

#### Section Used to Manually Obtain Latitude and Longitude for Hospitals
* no longer need to be run
---------------------------------------------------------------------------

In [22]:
# hosp_missing = hosp[hosp.latitude.isna()][['Facility Name','Address','City']].reset_index().copy()
# hosp_missing['latitude'] = [np.NaN for n in range(hosp_missing.shape[0])]
# hosp_missing['longitude'] = [np.NaN for n in range(hosp_missing.shape[0])]

# hosp_missing = pd.read_csv('temp_latlong_filling.csv', index_col='Facility ID')
# missing = hosp_missing['Facility Name']+', '+hosp_missing.Address+', '+hosp_missing.City+', USA'

In [4]:
# import geopy
# from geopy import Nominatim

# locator = Nominatim(user_agent="nycDSA")

In [420]:
# current = 88
# missing[current]

'William R Sharpe, Jr Hospital, 936 SHARPE HOSPITAL ROAD, WESTON, USA'

In [422]:
# lat = locator.geocode("William R. Sharpe Hospital, WESTON, USA")[1][0]
# long = locator.geocode("William R. Sharpe Hospital, WESTON, USA")[1][1]
# print(f'{lat}, {long}')

39.0408843, -80.4961853


In [423]:
# hosp_missing.loc[current,'latitude'] = lat
# hosp_missing.loc[current,'longitude'] = long

In [424]:
# hosp_missing.loc[[current]]

,Facility ID,Facility Name,Address,City,latitude,longitude
88,514012,"William R Sharpe, Jr Hospital",936 SHARPE HOSPITAL ROAD,WESTON,39.040884,-80.496185



---------------------------------------------------------------------------

## 🏥 Quick EDA

In [559]:
hosp.columns

Index(['Facility_Name', 'Address', 'City', 'State', 'ZIP_Code', 'County_Name',
       'Phone_Number', 'Hospital_Type', 'Hospital_Ownership',
       'Emergency_Services', 'Overall_Rating'],
      dtype='object')

In [513]:
print('-----------------------------------')
print(hosp.Hospital_Type.value_counts())
print('\n')
print('-----------------------------------')
print(hosp.Emergency_Services.value_counts())
print('\n')

-----------------------------------
Acute Care Hospitals                  553
Critical Access Hospitals              92
Psychiatric                            83
Acute Care - Department of Defense     12
Childrens                              11
Name: Hospital_Type, dtype: int64


-----------------------------------
Yes    640
No     111
Name: Emergency_Services, dtype: int64




#### Remove <span style="color:darkred">Psychiatric</span> & <span style="color:red">non-ER</span> Hospitals

In [514]:
hosp = hosp[(hosp.Hospital_Type!='Psychiatric') & (hosp.Emergency_Services=='Yes')]

In [517]:
print('-----------------------------------')
print(f'Total ZIP codes: {hosp.ZIP_Code.nunique()}')
print('\n')
print('-----------------------------------')
print('# Hospitals by State')
print(hosp.State.value_counts())
print('\n')
print('-----------------------------------')
print(hosp.Overall_Rating.value_counts())

-----------------------------------
Total ZIP codes: 611


-----------------------------------
# Hospitals by State
FL    170
GA    126
NC    103
VA     80
SC     55
WV     46
MD     43
DC      6
Name: State, dtype: int64


-----------------------------------
3                175
4                139
2                136
Not Available     94
5                 52
1                 33
Name: Overall_Rating, dtype: int64


## Stanley Martin Homes Target ZIP codes and STATES

In [522]:
smh = pd.read_csv('redfin_scrapy/smh_zipcodes.csv', index_col='ZipCode')
smh.State.value_counts()

FL    1464
VA    1213
NC    1079
GA     950
WV     851
MD     603
SC     534
DC     293
Name: State, dtype: int64

In [531]:
import json
with open('C:/Users/dn-83/Documents/bootcamp/capstone_proj/_data/dc_district_of_columbia_zip_codes_geo.min.json') as f:
    dc_geodata = json.load(f)

In [536]:
dc_geodata

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'STATEFP10': '11',
    'ZCTA5CE10': '20024',
    'GEOID10': '1120024',
    'CLASSFP10': 'B5',
    'MTFCC10': 'G6350',
    'FUNCSTAT10': 'S',
    'ALAND10': 6770280,
    'AWATER10': 2003315,
    'INTPTLAT10': '+38.8760098',
    'INTPTLON10': '-077.0253225',
    'PARTFLG10': 'N'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-77.064251, 38.888956],
       [-77.064311, 38.889104],
       [-77.064325, 38.889146],
       [-77.064354, 38.889255],
       [-77.064365, 38.889287],
       [-77.064377, 38.889325],
       [-77.064384, 38.88936],
       [-77.064393, 38.889408],
       [-77.064402, 38.889442],
       [-77.064409, 38.889473],
       [-77.064414, 38.889502],
       [-77.064418, 38.889535],
       [-77.064415, 38.889571],
       [-77.064409, 38.889598],
       [-77.064389, 38.889643],
       [-77.064378, 38.889659],
       [-77.064363, 38.889677],
       [-77.064339, 38.889712],
       [

In [557]:
import plotly.express as px
fig = px.choropleth(data_frame = smh[smh.State=='DC'], 
                    locations=smh[smh.State=='DC'].index,
                    geojson=dc_geodata, 
                    fitbounds='locations',
                    featureidkey='properties.ZCTA5CE10')

fig = px.scatter_mapbox(hosp[hosp.State=='DC'], lat="latitude", lon="longitude")

fig.update_layout(mapbox_style="open-street-map")
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

ValueError: Value of 'lat' is not the name of a column in 'data_frame'. Expected one of ['Facility_Name', 'Address', 'City', 'State', 'ZIP_Code', 'County_Name', 'Phone_Number', 'Hospital_Type', 'Hospital_Ownership', 'Emergency_Services', 'Overall_Rating'] but received: latitude

In [555]:
hosp[hosp.State=='DC']

,Facility_Name,Address,City,State,ZIP_Code,County_Name,Phone_Number,Hospital_Type,Hospital_Ownership,Emergency_Services,Overall_Rating
0,George Washington Univ Hospital,900 23RD ST NW,WASHINGTON,DC,20037,District of Columbia,(202) 716-4605,Acute Care Hospitals,Proprietary,Yes,1
2,Medstar Georgetown University Hospital,3800 RESERVOIR RD,WASHINGTON,DC,20007,District of Columbia,(202) 784-3000,Acute Care Hospitals,Proprietary,Yes,2
3,Sibley Memorial Hospital,5255 LOUGHBORO RD NW,WASHINGTON,DC,20016,District of Columbia,(202) 537-4680,Acute Care Hospitals,Voluntary non-profit - Other,Yes,4
4,United Medical Center,1310 SOUTHERN AVENUE SE,WASHINGTON,DC,20032,District of Columbia,(202) 574-6611,Acute Care Hospitals,Proprietary,Yes,1
5,Medstar Washington Hospital Center,110 IRVING STREET NW,WASHINGTON,DC,20010,District of Columbia,(202) 877-7000,Acute Care Hospitals,Voluntary non-profit - Other,Yes,3
6,Children'S National Hospital,"111 MICHIGAN AVE, NW",WASHINGTON,DC,20010,District of Columbia,(202) 476-5000,Childrens,Voluntary non-profit - Private,Yes,Not Available
